In [1]:
from palaven_api_helper import create_evaluation_session, fetch_instruction_dataset, fetch_evaluation_session
from palaven_api_helper import generate_prompt, augment_query
from palaven_api_helper import save_model_response, save_rag_model_response

import pandas as pd

dataset_id = 'f0444b12-5485-4299-b03b-3bdb6d4a2578'
evaluation_session = None
evaluation_session_id = '9DB25876-D92E-4CD0-9C20-7E6802387A5D'
batch_size = 20
llm = 'google-gemma-7b'
device_info = 'TPU v2'
tenant_id = '69A03A54-4181-4D50-8274-D2D88EA911E4'

In [2]:
if evaluation_session is None:
  evaluation_session = create_evaluation_session(dataset_id, batch_size, llm, device_info)
  evaluation_session_id = evaluation_session['sessionId']

In [2]:
print(evaluation_session_id)
print(evaluation_session)

9DB25876-D92E-4CD0-9C20-7E6802387A5D
None


In [3]:
data_shape = {
    'instruction_id': [],
    'instruction': [],
    'chat_completion': [],
    'elapsed_time': []
}

llm_responses_df = pd.DataFrame(data_shape)
llm_responses_df.set_index('instruction_id', inplace=True)

print(llm_responses_df)

Empty DataFrame
Columns: [instruction, chat_completion, elapsed_time]
Index: []


In [4]:
def add_instruction_to_df(instruction_id, instruction):
    llm_responses_df.loc[instruction_id, 'instruction'] = instruction
    llm_responses_df.loc[instruction_id, 'chat_completion'] = None
    llm_responses_df.loc[instruction_id, 'elapsed_time'] = None

In [5]:
instructions = fetch_instruction_dataset(evaluation_session_id, batch_number=1)

for item in instructions:
  add_instruction_to_df(item['instructionId'], item['instruction'])

print(llm_responses_df)

                                                      instruction  \
instruction_id                                                      
3877.0          ¿Quiénes están obligados al pago del impuesto ...   
3878.0          ¿Bajo qué circunstancias los residentes en el ...   
3879.0          ¿Qué ingresos deben declarar los residentes en...   
3880.0          ¿Qué diferencia hay en la obligación de pagar ...   
3881.0          ¿Qué se entiende por establecimiento permanent...   
3882.0          ¿Qué deben hacer los residentes en el extranje...   
3883.0          ¿Cuándo los residentes en el extranjero deben ...   
3884.0          ¿Por qué los residentes en el extranjero puede...   
3885.0          ¿Qué diferencia hay en la obligación de pagar ...   
3886.0          ¿Cuál es la diferencia en la obligación de pag...   
3887.0          ¿Quiénes están obligados al pago del impuesto ...   
3888.0          ¿Qué ingresos deben declarar los residentes en...   
3889.0          ¿Los residentes en

In [6]:
!pip install -U transformers
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 24.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 2.1 MB/s eta 0:00:00


In [7]:
from google.colab import userdata

hface_read_token = userdata.get('hface-read-token')

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it", token=hface_read_token)
model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it", device_map="auto", token=hface_read_token)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [9]:
import time

def get_chat_completion(instruction_prompt, instruction_id, instruction_count):

  start_time = time.time()

  tokenized_instruction = tokenizer(instruction_prompt, return_tensors="pt")
  chat_completion_result = model.generate(**tokenized_instruction, max_new_tokens=500)
  chat_completion = tokenizer.decode(chat_completion_result[0])

  end_time = time.time()
  elapsed_time = end_time - start_time

  print(f'{instruction_count} - LLM-ChatCompletion. InstructionId: {instruction_id},  Elapsed-Time: {elapsed_time:.2f} seconds')

  return chat_completion, elapsed_time

In [10]:
import numpy as np
import json

instruction_count = 1

for index, row in llm_responses_df.iterrows():
  instruction = llm_responses_df.loc[index, 'instruction']

  instruction_prompt = f"""
  <start_of_turn>user
  Answer the following question in a concise and informative manner. The question is written in Spanish language, then answer in Spanish language.
  {instruction}<end_of_turn>
  <start_of_turn>model
  """

  instruction_id = int(index)

  chat_completion, elapsed_time = get_chat_completion(instruction_prompt, instruction_id, instruction_count)

  save_model_response(evaluation_session_id, 1, instruction_count, instruction_id, chat_completion, elapsed_time)

  instruction_count += 1

1 - LLM-ChatCompletion. InstructionId: 3877,  Elapsed-Time: 41.96 seconds
1 - PalavenSaveResponse. InstructionId: 3877,  Elapsed-Time: 0.61 seconds
2 - LLM-ChatCompletion. InstructionId: 3878,  Elapsed-Time: 58.55 seconds
2 - PalavenSaveResponse. InstructionId: 3878,  Elapsed-Time: 0.30 seconds
3 - LLM-ChatCompletion. InstructionId: 3879,  Elapsed-Time: 25.25 seconds
3 - PalavenSaveResponse. InstructionId: 3879,  Elapsed-Time: 0.25 seconds
4 - LLM-ChatCompletion. InstructionId: 3880,  Elapsed-Time: 87.49 seconds
4 - PalavenSaveResponse. InstructionId: 3880,  Elapsed-Time: 0.33 seconds
5 - LLM-ChatCompletion. InstructionId: 3881,  Elapsed-Time: 47.10 seconds
5 - PalavenSaveResponse. InstructionId: 3881,  Elapsed-Time: 0.32 seconds
6 - LLM-ChatCompletion. InstructionId: 3882,  Elapsed-Time: 109.91 seconds
6 - PalavenSaveResponse. InstructionId: 3882,  Elapsed-Time: 0.33 seconds
7 - LLM-ChatCompletion. InstructionId: 3883,  Elapsed-Time: 62.29 seconds
7 - PalavenSaveResponse. InstructionI

In [12]:
from palaven_api_helper import post_request

def augment_query_v2(llm, tenant_id, instruction):
    palaven_base_url = "https://palaven-api-svc.azurewebsites.net/api"
    url = f'{palaven_base_url}/chatcompletion/{llm}/augmentquery'
    body = {
        'userId': tenant_id,
        'query': instruction
    }
    response = post_request(url, json=body)
    return response

In [13]:
import numpy as np
import json

instruction_count = 1

for index, row in llm_responses_df.iterrows():
  instruction = llm_responses_df.loc[index, 'instruction']

  query_augmentation_response = augment_query_v2(llm, tenant_id, instruction)
  instruction_prompt = query_augmentation_response['query']

  instruction_id = int(index)

  print(instruction_prompt)

  chat_completion, elapsed_time = get_chat_completion(instruction_prompt, instruction_id, instruction_count)
  save_rag_model_response(evaluation_session_id, 1, instruction_count, instruction_id, chat_completion, elapsed_time)

  instruction_count += 1

<start_of_turn>user
Answer the following question in a concise and informative manner following the next directions:
1. The question is in spanish, then answer in spanish.
2. Use one of the next articles to answer the given question. 
3. The articles are delimited by the tags <article></article>. Each article is different. You can generate an answer using one or many of the articles but in the response you must mention what articles you used.

<article>Artículo 1. Las personas físicas y las morales están obligadas al pago del impuesto sobre la renta en los siguientes casos:
I.
Las residentes en México, respecto de todos sus ingresos, cualquiera que sea la ubicación de la fuente de riqueza de donde procedan.
II.
Los residentes en el extranjero que tengan un establecimiento permanente en el país, respecto de los ingresos atribuibles a dicho establecimiento permanente.
III.
Los residentes en el extranjero, respecto de los ingresos procedentes de fuentes de riqueza situadas en territorio n